In [ ]:
from torch_geometric.datasets import Flickr
from torch_geometric.transforms import NormalizeFeatures, RandomNodeSplit
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from tqdm.notebook import tqdm
from utils import *
import pandas as pd
import multiprocessing as mp
from torch_geometric.utils import homophily
from tqdm.contrib.concurrent import process_map
from functools import partial
import pickle as pkl
import os

%load_ext autoreload
%autoreload 2


In [ ]:
with open('data/SBM/test.pkl', 'rb') as f:
    test = pkl.load(f)

In [ ]:
homophily(test.edge_index, test.y, method='node')

In [ ]:
G = to_networkx(test).to_undirected()

In [ ]:
def get_nbhd_weights(G, node, k, scheme='unif'):
    # Get dict containing nodes -> shortest path to node (i.e. depth).
    node_depth_map = pd.Series(nx.single_source_shortest_path_length(G, node, cutoff=k), name='distance')
    node_depth_map.index.name = 'node_id'
    node_depth_map = node_depth_map.drop(node) # Remove the node itself from list.
    node_depth_map = node_depth_map.reset_index()

    if scheme == 'geom':
        node_depth_map['weight'] = (0.5)**(node_depth_map['distance'] - 1)  # Weight =
    elif scheme == 'linear':
        node_depth_map['weight'] = 1 / node_depth_map['distance']
    else:
        node_depth_map['weight'] = 1
    return node_depth_map

In [ ]:
exp_path = 'experiments/16-03-2023_18-52-02_SBM'

preds_path = os.path.join(exp_path, 'preds.pkl')
with open(preds_path, 'rb') as f:
    preds = pkl.load(f)
preds = pd.DataFrame(preds)
test_x = pd.DataFrame(test.x.numpy())
test_y = pd.DataFrame(test.y.numpy())

print(len(preds))
print(len(test_x))
print(len(test_y))

In [ ]:
test_y.value_counts()

In [ ]:
probs = test_y.value_counts() / len(test_y)
probs.name = 'Proportion'
probs.index.name = 'Class'
probs
sum(probs**2)

In [ ]:
## Compute the NAPS prediction set for each node in advance.
alpha = 0.1
quantiles_nb = []
def calibrate_nbhd(node, scheme='unif'):
    nbs = get_nbhd_weights(G, node, k=2, scheme=scheme)
    nb_ids = nbs['node_id'].values
    weights = nbs['weight'].values
    quantile = calibrate_weighted(preds.loc[nb_ids].values,
                         np.squeeze(test_y.loc[nb_ids].values),
                                  weights, alpha)
    return {node: quantile}
# quantiles_nb = process_map(calibrate_nbhd, list(G.nodes), max_workers=12)

In [ ]:
def precompute_naps_sets(scheme):
    f = partial(calibrate_nbhd, scheme=scheme)
    quantiles_nb = process_map(f, list(G.nodes), max_workers=12)
    nz = [p for p in quantiles_nb if p is not None]
    res = {}
    for p in nz:
        res.update(p)
    nbhd_quantiles = pd.Series(res, name='quantile')
    nbhd_quantiles
    sets_nb = predict(preds.values, nbhd_quantiles.values[:, None])
    sets_nb = pd.Series(sets_nb, index=list(G.nodes), name='set')
    sets_nb = pd.DataFrame(sets_nb)
    sets_nb['set_size'] = sets_nb['set'].apply(len)
    sets_nb['covers'] = [test_y.loc[i].values in sets_nb.loc[i, 'set'] for i in sets_nb.index.values]
    return sets_nb

In [ ]:
naps_sets = precompute_naps_sets('unif')
napsl_sets = precompute_naps_sets('linear')
napsg_sets = precompute_naps_sets('geom')

In [ ]:
naps_sets.groupby('set_size').count()

In [ ]:
n_trials = 100
n_eval = 500
sccv_bins = [-1, 3, 9]
nodes = list(G.nodes())

In [ ]:
naps_stats = []
napsl_stats = []
napsg_stats = []
full_stats = []

# with mp.Pool(12) as p:
for k in tqdm(range(n_trials)):
    ## Sample the prediction nodes.
    pred_nodes = np.random.choice(nodes, size=n_eval, replace=False)
    # Neighbourhood calibration is pre-computed, so just get prediction sets for them.
    naps_stats.append(evaluate_predictions(naps_sets.loc[pred_nodes, 'set'].values,
                                         test_x.loc[pred_nodes].values,
                                         np.squeeze(test_y.loc[pred_nodes].values), alpha, sccv_bins
                                         ))
    napsl_stats.append(evaluate_predictions(napsl_sets.loc[pred_nodes, 'set'].values,
                                         test_x.loc[pred_nodes].values,
                                         np.squeeze(test_y.loc[pred_nodes].values), alpha, sccv_bins
                                         ))
    napsg_stats.append(evaluate_predictions(napsg_sets.loc[pred_nodes, 'set'].values,
                                         test_x.loc[pred_nodes].values,
                                         np.squeeze(test_y.loc[pred_nodes].values), alpha, sccv_bins
                                         ))

    # Full calibration
    quantile = calibrate(preds[~preds.index.isin(pred_nodes)].values,
                         np.squeeze(test_y[~test_y.index.isin(pred_nodes)].values), alpha)
    sets_full = predict(preds.loc[pred_nodes].values, quantile)
    full_stats.append(evaluate_predictions(sets_full,
                                           test_x.loc[pred_nodes].values,
                                           np.squeeze(test_y.loc[pred_nodes].values), alpha, sccv_bins))


In [ ]:
nb_df = pd.DataFrame(naps_stats, columns=['coverage', 'set_size', 'cc_set_size', 'sscv'])
nb_df['coverage'].plot(kind='hist', bins=30)
nb_df.describe()

In [ ]:
nb_df = pd.DataFrame(napsl_stats, columns=['coverage', 'set_size', 'cc_set_size', 'sscv'])
nb_df['coverage'].plot(kind='hist', bins=30)
nb_df.describe()

In [ ]:
nb_df = pd.DataFrame(napsg_stats, columns=['coverage', 'set_size', 'cc_set_size', 'sscv'])
nb_df['coverage'].plot(kind='hist', bins=30)
nb_df.describe()


In [ ]:
full_df = pd.DataFrame(full_stats, columns=['coverage', 'set_size', 'cc_set_size', 'sscv'])
full_df['coverage'].plot(kind='hist', bins=30)
full_df.describe()

In [ ]:
from random import sample
## Split the test nodes into non-overlapping neighbourhoods
def split_into_neighbourhoods(test_nodes):
    test_subgraph = G.subgraph(test_nodes).copy()
    nbhds = []
    while test_subgraph.number_of_nodes() > 0:
        root = sample(list(test_subgraph.nodes()), 1)[0]
        nbhd_nodes = list(nx.single_source_shortest_path_length(test_subgraph, root, cutoff=2).keys())
        nbhds.append(nbhd_nodes)
        test_subgraph.remove_nodes_from(nbhd_nodes)
    return sorted(nbhds, key=lambda x: len(x), reverse=True)[:10]

In [ ]:
aps_nccvs = []
naps_nccvs = []
napsl_nccvs = []
napsg_nccvs = []
for k in tqdm(range(n_trials)):
    np.random.shuffle(nodes)
    n_calib = len(nodes) // 2
    calib_nodes = nodes[:n_calib]
    test_nodes = nodes[n_calib:]

    ## Calibrate the regular CP on calibration nodes and make predictions on test nodes
    quantile = calibrate(preds.loc[calib_nodes].values,
                         np.squeeze(test_y.loc[calib_nodes].values), alpha)
    sets_full = pd.Series(predict(preds.loc[test_nodes].values, quantile), index=test_nodes)
    nbhds = split_into_neighbourhoods(test_nodes)
    aps_nb_coverages = []
    naps_nb_coverages = []
    napsl_nb_coverages = []
    napsg_nb_coverages = []

    for nbhd in nbhds:
        aps_nb_coverages.append(np.mean([test_y.loc[node].item() in sets_full[node] for node in nbhd]))
        naps_nb_coverages.append(naps_sets.loc[nbhd, 'covers'].mean())
        napsl_nb_coverages.append(napsl_sets.loc[nbhd, 'covers'].mean())
        napsg_nb_coverages.append(napsg_sets.loc[nbhd, 'covers'].mean())

    aps_nccv = max(np.abs(np.array(aps_nb_coverages) - (1 - alpha)))
    naps_nccv = max(np.abs(np.array(naps_nb_coverages) - (1 - alpha)))
    napsl_nccv = max(np.abs(np.array(napsl_nb_coverages) - (1 - alpha)))
    napsg_nccv = max(np.abs(np.array(napsg_nb_coverages) - (1 - alpha)))

    aps_nccvs.append(aps_nccv)
    naps_nccvs.append(naps_nccv)
    napsl_nccvs.append(napsl_nccv)
    napsg_nccvs.append(napsg_nccv)


In [ ]:
pd.DataFrame(aps_nccvs).describe()

In [ ]:
pd.DataFrame(naps_nccvs).describe()

In [ ]:
pd.DataFrame(napsl_nccvs).describe()

In [ ]:
pd.DataFrame(napsg_nccvs).describe()